<a href="https://colab.research.google.com/github/Smart1mushtak/diabetic/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## basic 
## sentiment analysis

In [3]:
import nltk

In [9]:
!pip install contractions 
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 9.4 MB/s eta 0:00:00


In [40]:
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from contractions import fix
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

In [41]:
data = pd.read_csv("sentidata.csv",error_bad_lines=False, engine="python")

<ipython-input-41-1317fba31049>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("sentidata.csv",error_bad_lines=False, engine="python")
Skipping line 23927: unexpected end of data


In [42]:
data.head(2)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0


In [43]:
#  preporcessing

# 1.remove extra space,newline
# 2.contraction mapping
# 3.handeling accented
# 4.cleaning text tokenization,lowercase conversion,remove punction,remove stopwords len(word)<2 remove
# 5. lemmatization

In [44]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
def remove_blank(data):
    formatted_text = data.replace('\\n',' ').replace('\t',' ')
    return formatted_text

def expand_text(data):
    fixed_text = fix(data)
    return fixed_text

def handle_accented(data):
    fixed_text = unidecode(data)
    return fixed_text

stopword_list = stopwords.words("english")
stopword_list.remove("no")
stopword_list.remove("nor")
stopword_list.remove("not")

def clean_text(data):
    tokens = word_tokenize(data)
    clean_data = [word.lower() for word in tokens if (word.lower() not in punctuation) and (word.lower() not in stopword_list) and ( len(word)>2) and (word.isalpha()) ]
    return clean_data

def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    final_text = []
    for word in data:
        lemmatized_word = lemmatizer.lemmatize(word)
        final_text.append(lemmatized_word)
    return " ".join(final_text)

In [65]:
x_train,x_test,y_train,y_test=train_test_split(data.text,data.label,test_size=0.25,random_state=45)

In [66]:
from tqdm import tqdm 
import pandas 
tqdm.pandas()

In [67]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [70]:
clean_train = x_train.progress_apply(remove_blank)
clean_test = x_test.progress_apply(remove_blank)

clean_train = clean_train.progress_apply(expand_text)
clean_test = clean_test.progress_apply(expand_text)

clean_train = clean_train.progress_apply(handle_accented)
clean_test = clean_test.progress_apply(handle_accented)

clean_train = clean_train.progress_apply(clean_text)
clean_test = clean_test.progress_apply(clean_text)

clean_train = clean_train.progress_apply(lemmatization)
clean_test = clean_test.progress_apply(lemmatization)

100%|██████████| 5982/5982 [00:03<00:00, 1950.50it/s]


In [71]:
clean_train

8252     movie complete utter waste time one worst film...
3411     know chill will usually played lovable old sor...
19927    not movie would typically watch morning got co...
14995    summary not worth film avid gone wind fan disa...
740      nothing sitcom seinfeld eccentric hilarious ch...
                               ...                        
16317    really mean caught last night not expecting go...
12895    one funniest movie ever saw old boy jason shep...
6012     friend film like bete aka beast monstro done o...
6558     ridiculous fluff compound error trying meaning...
23499    not expecting much film eager try something in...
Name: text, Length: 17943, dtype: object

In [72]:
clean_test

4870     always difficult bring page book three hour fi...
21850    saw movie first thought boring nothing happeni...
22136    though derivative labyrinth still stand highli...
3811     know everyone said movie utter crap agree spen...
19343    see countless preview trailer enjoyed knoxvill...
                               ...                        
7131     retrospect golden era american cinema demonstr...
14527    mind best ealing comedy one favourite film tim...
1515     great battle finale nice set help keep movie e...
15059    saw night demon first saw night demon unfortun...
14371    year since movie made romance mysticism two mo...
Name: text, Length: 5982, dtype: object

In [73]:
# Count Vectorizer 
count = CountVectorizer(max_df=0.95,max_features=1000,min_df=2)
# max_df = 0.95  : 100 95> 
# min_df = 1,2,3 
# max_features 

In [74]:
count_train = count.fit_transform(clean_train)
count_test = count.transform(clean_test)

In [76]:
pd.DataFrame(count_train.A,columns=count.get_feature_names_out())

,ability,able,absolutely,accent,across,act,acted,acting,action,actor,...,wrong,wrote,yeah,year,yes,yet,york,young,younger,zombie
0,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,3,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17938,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17939,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
17940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17941,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,1,1,0


In [77]:
mnb = MultinomialNB()
mnb.fit(count_train.A,y_train)

MultinomialNB()

In [78]:
predict_y = mnb.predict(count_test.A)

In [79]:
accuracy_count = accuracy_score(y_test,predict_y)*100
accuracy_count

83.86827148111

In [80]:
# Tfidf 
# Count Vectorizer 
tfidf = TfidfVectorizer(max_df=0.95,max_features=1000,min_df=2)
# max_df = 0.95  : 100 95> 
# min_df = 1,2,3 
# max_features 

In [81]:
tfidf_train = tfidf.fit_transform(clean_train)
tfidf_test = tfidf.transform(clean_test)


In [82]:
pd.DataFrame(tfidf_train.A,columns= tfidf.get_feature_names_out())

,ability,able,absolutely,accent,across,act,acted,acting,action,actor,...,wrong,wrote,yeah,year,yes,yet,york,young,younger,zombie
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.056849,0.0,0.000000,...,0.000000,0.111152,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,0.123757,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.119829,0.000000,0.0,0.000000,0.000000,0.0
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.124144,...,0.000000,0.000000,0.0,0.127177,0.000000,0.000000,0.0,0.153453,0.000000,0.0
3,0.0,0.083392,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,0.0,0.000000,0.067451,0.0,0.0,0.0,0.0,0.000000,0.0,0.135518,...,0.066182,0.000000,0.0,0.000000,0.000000,0.058425,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17938,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
17939,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.044845,0.058113,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
17940,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
17941,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.091307,0.000000,0.057639,0.0,0.055086,0.085481,0.0


In [83]:
mnb = MultinomialNB()
mnb.fit(tfidf_train.A,y_train)
predict_y = mnb.predict(tfidf_test.A)
accuracy_tfidf = accuracy_score(y_test,predict_y)*100
accuracy_tfidf

84.16917418923437